In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import urllib
import time
from selenium.webdriver.chrome.options import Options
import selenium
from selenium import webdriver as wb
from selenium.webdriver.common.keys import Keys
from lxml import html
import re
from datetime import datetime, timedelta, date
from IPython.display import clear_output
import pickle
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
import logging
import io
import traceback
import winsound
import warnings
warnings.filterwarnings("ignore")

In [2]:
def tables(url,s):
    try:
        parameter = f"&tab=form_teams&cmp=1&lm={s}"
        r= requests.get(url+parameter)
        soup = BeautifulSoup(r.text,"html.parser")
        stats =pd.read_html(url+parameter,flavor='lxml')
        t1data = stats[2].T.reset_index(drop=True).iloc[0].append(stats[2].T.reset_index(drop=True).iloc[2]).reset_index(drop=True)
        t1col = ('Home ' + stats[2].T.reset_index(drop=True).iloc[1]).append('Away ' + stats[2].T.reset_index(drop=True).iloc[1])
        t2data = stats[3].T.reset_index(drop=True).iloc[0].append(stats[3].T.reset_index(drop=True).iloc[2]).reset_index(drop=True)
        t2col = ('Home ' + stats[3].T.reset_index(drop=True).iloc[1]).append('Away ' + stats[3].T.reset_index(drop=True).iloc[1])
        col = t1col.append(t2col)
        datas = t1data.append(t2data).reset_index(drop=True)
        df = pd.DataFrame(columns=col,data = np.array([datas])).add_prefix(f'{s}_')
        return df
    except:
        return pd.DataFrame()

def yarat(url):
    try:
        parameter = "&tab=form_teams&cmp=1&lm=5"
        r= requests.get(url+parameter)
        soup = BeautifulSoup(r.text,"html.parser")
        try:
            evname = soup.find_all('div',{"class":"live_game_ht"})[0].text.replace('\n','')
        except:
            evname = np.nan
        try:
            evid = soup.find_all('div',{"class":"live_game_ht"})[0].find('a')['href'].split('/')[-2]
        except:
            evid = np.nan
        try:
            depid = soup.find_all('div',{"class":"live_game_at"})[0].find('a')['href'].split('/')[-2]
        except:
            depid = np.nan
        try:
            depname = soup.find_all('div',{"class":"live_game_at"})[0].text.replace('\n','')
        except:
            depname = np.nan
        try:
            evgoal = int(soup.find_all('div',{"class":"live_game_goal"})[0].text.replace('\n',''))
        except:
            evgoal = np.nan
        try:    
            depgoal = int(soup.find_all('div',{"class":"live_game_goal"})[1].text.replace('\n',''))
        except:
            depgoal = np.nan
        try:
            league = soup.find_all('div',{"class":"breadcrumb"})[0].find_all('a',href=True)[2]['href'].split('/')[-2]
        except:
            league = np.nan
        try:
            text = soup.find_all('div',{"class":"block_header bkcenter"})[0].text.strip('\n')
        except:
            text = np.nan

        data = {'Home Team':evname,'Home ID':evid,'Away Team':depname,'Away ID':depid,'Home Goal':evgoal,'Away Goal':depgoal,'League':league,'Text':text,

    }
        datas = pd.DataFrame()
        for i in [1,2,3,4,5,10,15]:
            datas = pd.concat([datas,tables(url,i)],axis=1)
        return pd.concat([pd.DataFrame(data,index=[0]),datas],axis=1)
    except:
        return pd.DataFrame()
    
def hreftopla(url):
    hrefs = []
    start = 'http://soccer365.me'
    r= requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    for a in soup.find_all('a',class_='game_link',href=True):
        hrefs.append(start +a['href'])
    return hrefs

In [6]:
%%time
yarat(matchlinks[0])

Wall time: 12 s


,Home Team,Home ID,Away Team,Away ID,Home Goal,Away Goal,League,Text,1_Home Games,1_Home Wins,...,15_Away Corner,15_Away Corner (competitor),15_Away Fouls,15_Away Fouls (competitor),15_Away Offsides,15_Away Offsides (competitor),15_Away Yellow cards,15_Away Yellow cards (competitor),15_Away Red cards,15_Away Red cards (competitor)
0,Slavia Prague,149,Legia,622,2,2,20,"UEFA Europa League, play-off round, 19.08.2021...",1,100% 1,...,5.0,4.13,11.87,15.0,1.0,0.87,2.2,2.13,0.13,0.13


In [4]:
# txt file to list
with open('2000mac.txt', 'r') as f:
    matchlinks = [line.strip() for line in f]

In [4]:
len(matchlinks)

436692

In [5]:
from pqdm.threads import pqdm

In [13]:
pd.concat(pqdm(matchlinks[:24],yarat,n_jobs=100),ignore_index=True)

SUBMITTING | :   0%|          | 0/24 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/24 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/24 [00:00<?, ?it/s]

,Home Team,Home ID,Away Team,Away ID,Home Goal,Away Goal,League,Text,2_Home Games,2_Home Wins,...,1_Away Corner,1_Away Corner (competitor),1_Away Fouls,1_Away Fouls (competitor),1_Away Offsides,1_Away Offsides (competitor),1_Away Yellow cards,1_Away Yellow cards (competitor),1_Away Red cards,1_Away Red cards (competitor)
0,Omonia Nicosia,8324,Antwerp,8856,4,2,20,"UEFA Europa League, play-off round, 19.08.2021...",2,100% 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Mura,655,Sturm Graz,547,1,3,20,"UEFA Europa League, play-off round, 19.08.2021...",2,50% 1,...,1.0,4.0,17.0,12.0,3.0,1.0,3.0,3.0,0.0,0.0
2,Olympiakos,113,Slovan Bratislava,563,3,0,20,"UEFA Europa League, play-off round, 19.08.2021...",2,0,...,7.0,3.0,8.0,12.0,2.0,1.0,0.0,3.0,0.0,0.0
3,Rapid Vienna,631,Zarya,293,3,0,20,"UEFA Europa League, play-off round, 19.08.2021...",2,0,...,6.00,1.00,13.00,7.00,2.00,1.00,?,?,?,?


In [ ]:
eskiveriler=pd.DataFrame()
sonlen=0
ilklen=0
for i in range(0,2000):
    sonlen = ilklen
    ilklen = len(eskiveriler)
    print(f'Başarılı Eklenen Veri {ilklen-sonlen} / {100}')
    print(f'{i+1}/{2000} : {((i+1)/2000)*100}%')
    maclardf = pd.concat(pqdm(matchlinks[i*100:(i+1)*100],yarat,n_jobs=24),ignore_index=True)
    eskiveriler=eskiveriler.append(maclardf,ignore_index=True)
    if i%10 == 0:
        eskiveriler.to_csv('train.csv')
    clear_output(wait=True)
eskiveriler.to_csv('train.csv')

Başarılı Eklenen Veri 55 / 100
2/2000 : 0.1%


SUBMITTING | :   0%|          | 0/100 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
maclardf = pd.concat(pqdm(merged[250*100:],yarat,n_jobs=24),ignore_index=True)
eskiveriler=eskiveriler.append(maclardf,ignore_index=True)
eskiveriler.to_csv('train.csv')

SUBMITTING | :   0%|          | 0/764 [00:00<?, ?it/s]

PROCESSING | :   0%|          | 0/764 [00:00<?, ?it/s]

COLLECTING | :   0%|          | 0/764 [00:00<?, ?it/s]

In [5]:
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count

In [27]:
if __name__ == '__main__':
    pool = Pool(cpu_count() * 2)
    for i in matchlinks[:100]:
        results = pool.map(yarat,i)

In [26]:
matchlinks[99]

'http://soccer365.me/games/1589498/'

In [10]:
from bs4 import BeautifulSoup
import requests
from datetime import date, timedelta
from multiprocessing.dummy import Pool
import tqdm

In [5]:
import os, glob
import pandas as pd
from multiprocessing import Pool

In [ ]:
data = None
if __name__ == '__main__':
    p = Pool(6)
    data = pd.concat(p.map(yarat, matchlinks[:20])).reset_index(drop=True)